## INTRO

nro id : se não tiver length 16 adicionar 0 na frente

instituição
- hospital : não é de ensino, tem q marcar q não é de academia
- mesma instituição, registros e tempos diferentes  =>  tem q agrupar
similaridade de string -> função cosseno de PLN (ngramas)



1) pegar o nome de todas as instituições do brasil (usar a lista do MEC)


- fazer a tabela de frequência /
eliminar oq tem pouca frequência
- se for secretaria municipal algo assim e aparecer poucas vezes => só rotular como servidor público
- se estiver so EACH (sem ter USP no nome), categorizar como 'outros'
- lista de universidades
    - tabela do ROR
    - tabela do MEC

In [ ]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

## READ_CSV

In [ ]:
df = pd.read_csv('atuacoes_2.csv', encoding='UTF-8', nrows=50) #, dtype=str, nrows=1000)

df['ANO-FIM'] = df['ANO-FIM'].fillna(2024)
df['FLAG-DEDICACAO-EXCLUSIVA'] = df['FLAG-DEDICACAO-EXCLUSIVA'].replace('NAO', False).replace('SIM', True)

df = df.astype({
    'NRO-ID-CNPQ': int,
    'CODIGO-INSTITUICAO': str,
    'NOME-INSTITUICAO': str,
    'ANO-INICIO': int,
    'ANO-FIM': int,
    'ENQUADRAMENTO-FUNCIONAL': str,
    'TIPO-DE-VINCULO': str,
    'OUTRAS-INFORMACOES': str,
    'CARGA-HORARIA-SEMANAL': float,
    'FLAG-DEDICACAO-EXCLUSIVA': bool
})

df = df.fillna(' ')
df2 = df.copy()

#df

In [ ]:
df2 = df.copy(deep=True)
df2.isna().sum()

In [ ]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')
df_uni_filtered = df_uni_filtered.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_filtered

## DATA PREPROCESSING

In [ ]:
def clean_names(df, column_name):

    # Remover NaN
    #df.dropna(subset=[column_name])

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace
    df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [ ]:
# Lista de Curriculo Lattes

# Tratar os nomes da NOME-INSTITUICAO
df2 = df2.apply(lambda x: x.astype(str).str.upper())
df2 = clean_names(df2, 'NOME-INSTITUICAO')

# Dropar linhas com IES < 50
#df2 = df2[df2.groupby('NOME-INSTITUICAO')['NOME-INSTITUICAO'].transform('count').ge(50)]

# Split df: educacao & outros
df2_edu = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['LIVRE', 'COLABORADOR', 'PROFESSOR_VISITANTE'])].copy(deep=True)
df2_outros = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['SERVIDOR_PUBLICO', 'CELETISTA', 'NAN', 'OUTRO'])].copy(deep=True)

df2_edu

In [ ]:
print(df2_edu['NOME-INSTITUICAO'].isna().sum())
print(df2_edu['NOME-INSTITUICAO'].value_counts())

In [ ]:
# Lista de IES

df_uni_filtered = df_uni_filtered.apply(lambda x: x.astype(str).str.upper())
df_uni_filtered = clean_names(df_uni_filtered, 'NOME_DA_IES')

# merge colunas NOME e SIGLA
df_uni_filtered['NOME_E_SIGLA'] = df_uni_filtered['SIGLA'] + ' ' + df_uni_filtered['NOME_DA_IES']
df_uni_filtered

## NLP


### Cosine

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

threshold = 0.8

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES'])

df2_edu['nome_corrigido'] = ''

for index, row in df2_edu.iterrows():

    nome_instituicao = row['NOME-INSTITUICAO']

    vector_comparacao = vectorizer.transform([nome_instituicao])

    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_universidades)

    max_value = np.max(similaridade)
    max_index = np.argmax(similaridade)

    nome_corrigido = df_uni_filtered.iloc[max_index]['NOME_DA_IES']

    if (max_value >= threshold):
        df2_edu.at[index, 'nome_corrigido'] = nome_corrigido
    else:
        df2_edu.at[index, 'nome_corrigido'] = nome_instituicao

In [ ]:
# oq foi alterado
pd.set_option('display.max_rows', 10)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido']])
pd.set_option('display.max_rows', 6)

In [ ]:
# SIMILARIDADE POR NOME E SIGLA
# EM QUANTIDADE MAIOR DE AMOSTRA, NAO FUNCIONA MUITO BEM

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

threshold = 0.8

vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_E_SIGLA'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_E_SIGLA'])

df2_edu['nome_corrigido'] = ''

for index, row in df2_edu.iterrows():

    nome_instituicao = row['NOME-INSTITUICAO']

    vector_comparacao = vectorizer.transform([nome_instituicao])

    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_universidades)

    max_value = np.max(similaridade)
    max_index = np.argmax(similaridade)

    nome_corrigido = df_uni_filtered.iloc[max_index]['NOME_DA_IES']

    if (max_value >= threshold):
        df2_edu.at[index, 'nome_corrigido'] = nome_corrigido
    else:
        df2_edu.at[index, 'nome_corrigido'] = nome_instituicao

### Cosine by N-gram


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

threshold = 0.6

vectorizer = TfidfVectorizer(ngram_range=(2, 3)).fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES'])

df2_edu['nome_corrigido_ngram'] = ''

for index, row in df2_edu.iterrows():
    nome_instituicao = row['NOME-INSTITUICAO']

    vector_comparacao = vectorizer.transform([nome_instituicao])

    similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_universidades)

    max_value = np.max(similaridade)  # Valor máximo de similaridade
    max_index = np.argmax(similaridade)  # Índice do valor máximo

    nome_corrigido = df_uni_filtered.iloc[max_index]['NOME_DA_IES']

    if max_value >= threshold:
        df2_edu.at[index, 'nome_corrigido_ngram'] = nome_corrigido
    else:
        df2_edu.at[index, 'nome_corrigido_ngram'] = nome_instituicao

In [ ]:
pd.set_option('display.max_rows', 100)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido_ngram']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido_ngram']])

## Jaccard

In [ ]:
import numpy as np

def jaccard_similarity(str1, str2):

    set1 = set(str1.split())
    set2 = set(str2.split())

    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))

    return intersection / union if union != 0 else 0

threshold = 0.5

df2_edu['nome_corrigido'] = ''

for index, row in df2_edu.iterrows():

    nome_instituicao = row['NOME-INSTITUICAO']
    max_similarity = 0
    nome_corrigido = nome_instituicao

    # TODO: deixar a set previamente pronta
    for idx_uni, row_uni in df_uni_filtered.iterrows():
        nome_uni = row_uni['NOME_DA_IES']
        similarity = jaccard_similarity(nome_instituicao, nome_uni)

        if similarity > max_similarity:
            max_similarity = similarity
            nome_corrigido = nome_uni

    if max_similarity >= threshold:
        df2_edu.at[index, 'nome_corrigido'] = nome_corrigido
    else:
        df2_edu.at[index, 'nome_corrigido'] = nome_instituicao


In [ ]:
# oq foi alterado
pd.set_option('display.max_rows', 10)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido']])
pd.set_option('display.max_rows', 6)

## Euclidean Distance

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

threshold = 0.8

vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES'])

df2_edu['nome_corrigido'] = ''

for index, row in df2_edu.iterrows():
    nome_instituicao = row['NOME-INSTITUICAO']

    vector_comparacao = vectorizer.transform([nome_instituicao])

    distancias = euclidean_distances(vector_comparacao, tfidf_matrix_universidades)

    min_distancia = np.min(distancias)
    min_index = np.argmin(distancias)

    nome_corrigido = df_uni_filtered.iloc[min_index]['NOME_DA_IES']

    if min_distancia <= threshold:
        df2_edu.at[index, 'nome_corrigido'] = nome_corrigido
    else:
        df2_edu.at[index, 'nome_corrigido'] = nome_instituicao

### Clustering: K-Means
- ARRUMAR

In [ ]:
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_E_SIGLA'].values)
tfidf_matrix_1 = vectorizer.transform(df_uni_filtered['NOME_E_SIGLA'].values)

n_clusters = 2754  # quantidade de IES existentes
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(tfidf_matrix_1)

cluster_labels = kmeans.labels_

for i in range(n_clusters):
    cluster_points = np.where(cluster_labels == i)[0]
    cluster_center = kmeans.cluster_centers_[i]

    # Find document closest to the centroid (most recurring)
    closest_doc_idx = np.argmax(cosine_similarity(tfidf_matrix_1[cluster_points], [cluster_center]))
    most_recurring_idx = cluster_points[closest_doc_idx]
    original_row_index = df2_edu.index[most_recurring_idx]
    most_recurring_document = df2_edu.loc[original_row_index]
    print(most_recurring_document['NOME-INSTITUICAO'])
most_recurring_document

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES'])

num_clusters = 4000 # qtd de IES
kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(tfidf_matrix_universidades)
clusters = kmeans.predict(tfidf_matrix_universidades)

df_uni_filtered['cluster'] = clusters

# Crie um dicionário para armazenar o nome mais representativo de cada cluster
representative_names = {}
for cluster in range(num_clusters):
    cluster_names = df_uni_filtered[df_uni_filtered['cluster'] == cluster]['NOME_DA_IES']
    # Escolha o nome mais frequente no cluster como representante
    representative_names[cluster] = cluster_names.mode()[0]

# Nova coluna
df2_edu['nome_corrigido_kmeans'] = ''

for index, row in df2_edu.iterrows():
    nome_instituicao = row['NOME-INSTITUICAO']

    vector_comparacao = vectorizer.transform([nome_instituicao])

    # Prever o cluster para o nome da instituição
    cluster_prediction = kmeans.predict(vector_comparacao)[0]

    # Obter o nome mais representativo do cluster
    nome_corrigido = representative_names.get(cluster_prediction, nome_instituicao)

    df2_edu.at[index, 'nome_corrigido_kmeans'] = nome_corrigido

    # print(f'{nome_instituicao} corrigido para {nome_corrigido}')


In [ ]:
pd.set_option('display.max_rows', 100)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido_kmeans']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido_kmeans']])

### Word Embedding
- ARRUMAR
- muito lentooo
- 50k linhas : 3min

In [ ]:
!python -m spacy download pt_core_news_sm
import spacy

# AJUSTAR O CODIGO PQ NAO TROCA A COLUNA NEM NADA

nlp = spacy.load("pt_core_news_sm")  # Carrega um modelo pré-treinado
df2['spacy_similarity'] = df2.apply(
    lambda row: nlp(row['NOME-INSTITUICAO']).similarity(nlp(df_uni_filtered['NOME_DA_IES'][row['indice_mais_similar']])),
    axis=1
)

df2

### Outras possibilidades:
- Word2Vect
- Modelos que levam em consideracao o sentido semantico

## MERGE SAME UNI

In [ ]:
# merge ANO-INICIO e ANO-FIM com mesmo CODIGO-INSTITUICAO e NRO-ID-CNPQ

df_merged = df.groupby(['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO']).agg(
    earliest_date1=('ANO-INICIO', 'min'),
    latest_date2=('ANO-FIM', 'max')
).reset_index()


# df.loc[df['NRO-ID-CNPQ'] == 3300778291054405]

In [ ]:
# merge colunas NOME-INSTITUICAO com df_merged

df_drop_duplicates = df.drop_duplicates(subset=['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO'])
df_inst_merged = pd.merge(df_merged, df_drop_duplicates, on=['CODIGO-INSTITUICAO', 'NRO-ID-CNPQ'], how='left')
df_inst_merged['NOME-INSTITUICAO'] = df_inst_merged['NOME-INSTITUICAO'].str.upper()

df_inst_merged = df_inst_merged[['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO', 'earliest_date1', 'latest_date2', 'NOME-INSTITUICAO']]
df_inst_merged

## EXPORT TO CSV

In [ ]:
df_inst_merged.to_csv("atuacoes_2_filtrado.csv", encoding='utf8')